Inference\
https://wikidocs.net/115055

https://huggingface.co/docs/transformers/index

http://kkma.snu.ac.kr/documents/index.jsp?doc=postag


# Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
# Date; 2. 10. 2022 - 2. 22. 2022
# Title: Korean_NER
# Professor: Seung-Hoon Na

In [1]:
import os
import pandas as pd
import json
import transformers
from transformers import BertTokenizer
# from tqdm.auto import tqdm

In [2]:
PATH_dir = 'C:\\Users\\LG\\Desktop\\github\\JBNU-2022-SPRING\\English world class tagging & Korean_Named Entity Recognition\\Ko_En_NER_POStag_data\Ko_NER_POS'
file_name_ko_train = 'train.txt'
file_name_ko_test = 'test.txt'
file_name_ko_dev = 'dev.txt'

# temp

In [3]:
class Data_prepro:
    
    def make_json(file_name):
        PATH_data = os.path.join(PATH_dir, file_name)
        prepro_file_name = 'prepro_' + file_name.rsplit('.')[0] + '.json' #change file extention from '.txt' to '.json'
        PATH_preprodata = os.path.join(PATH_dir, prepro_file_name)
        
    def read_file(path):
        with open(PATH_data, 'r', encoding='UTF8') as f:
            rawdata = f.readlines()
            
        return rawdata
        
    def Data_NER(path):
        
        # data preprocessing by using pandas
        df0 = pd.DataFrame(rawdata)
        df1 = pd.DataFrame(df0[0].str.split('/').tolist(),columns=['1-1','1-2'])
        df2 = pd.DataFrame(df0[0].str.split('\t').tolist(),columns=['1','2','3','4'])
        df_final = pd.concat([df1['1-1'],df2['4'].str.strip('\n')],axis = 1)
        df_final = pd.DataFrame(df_final)
        df_final = df_final.rename(columns={'1-1':'Entity','4':'tag'})
        
        PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
        tokenizer = BertTokenizer.from_pretrained(PRETAINED_MODEL_NAME)
        
        # make dataset & tokenizer_tagging [original, token, tokenizer_tagging]
        lst=[]
        lst_set = []
        lst_ent = []
        lst_tag = []
        for i in range(6945):
            if df_final['tag'][i] != None:
                morph_to_tokens = tokenizer.tokenize(df_final['Entity'][i])
                if len(df_final['Entity'][i]) == len(morph_to_tokens[0]):
                    lst_ent.append(df_final['Entity'][i])
                    lst_tag.append(df_final['tag'][i])

                else:
                    if df_final['tag'][i] == 'O':
                        lst_ent.append(df_final['Entity'][i])
                        lst_tag.append(df_final['tag'][i])
                        for i in range(len(morph_to_tokens)-1):
                            lst_tag.append(df_final['tag'][i])

                    else:
                        lst_ent.append(df_final['Entity'][i])
                        lst_tag.append(df_final['tag'][i])
                        split = df_final['tag'][i].split('-')
                        for i in range(len(morph_to_tokens)-1):
                            results = 'I-' + split[-1]
                            lst_tag.append(results)

            else:
                lst_ent = result = ' '.join(str(s) for s in lst_ent)
                lst_set.append(lst_ent)
                morph_to_tokens = tokenizer.tokenize(lst_ent)
                lst_set.append(morph_to_tokens)
                lst_set.append(lst_tag)
                lst.append(lst_set)
                result = 0
                lst_ent = []
                lst_tok = []
                lst_tag = []
                lst_set = []
            
        return lst
    
    
    def rewrite_json():
        with open(PATH_preprodata, 'w') as f:
            json.dump(lst, f)
            
        with open(PATH_preprodata, 'r') as f:
            preprodata = json.load(f)    

# ======================================

In [11]:
def make_json(file_name):
    global df_final
    PATH_data = os.path.join(PATH_dir, file_name)

    #change file extention from '.txt' to '.json'
    prepro_file_name = 'prepro_' + file_name.rsplit('.')[0] + '.json'
    PATH_preprodata = os.path.join(PATH_dir, prepro_file_name)

    with open(PATH_data, 'r', encoding='UTF8') as f:
        rawdata = f.readlines()

    # data preprocessing by using pandas
    df0 = pd.DataFrame(rawdata)
    
    if file_name == 'dev.txt':
        df1 = pd.DataFrame(df0[0].str.split('/').tolist(),columns=['1-1','1-2'])
        df2 = pd.DataFrame(df0[0].str.split('\t').tolist(),columns=['1','2','3','4'])
        df_final = pd.concat([df1['1-1'],df2['4'].str.strip('\n')],axis = 1)
        df_final = pd.DataFrame(df_final)
        df_final = df_final.rename(columns={'1-1':'Entity','4':'tag'})
    else:
        df1 = pd.DataFrame(df0[0].str.split('/').tolist(),columns=['1-1','1-2','1-3'])
        df2 = pd.DataFrame(df0[0].str.split('\t').tolist(),columns=['1','2','3','4'])
        df_final = pd.concat([df1['1-1'],df2['4'].str.strip('\n')],axis = 1)
        df_final = pd.DataFrame(df_final)
        df_final = df_final.rename(columns={'1-1':'Entity','4':'tag'})
   
    # upload tokenizer
    PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
    tokenizer = BertTokenizer.from_pretrained(PRETAINED_MODEL_NAME)

    # make dataset & tokenizer_tagging [original, token, tokenizer_tagging]
    lst=[]
    lst_set = []
    lst_ent = []
    lst_tag = []
    for i in range(len(df_final)):
        if df_final['tag'][i] != None:
            morph_to_tokens = tokenizer.tokenize(df_final['Entity'][i])
            if 1 == len(morph_to_tokens):
                lst_ent.append(df_final['Entity'][i])
                lst_tag.append(df_final['tag'][i])

            else:
                if df_final['tag'][i] == 'O':
                    lst_ent.append(df_final['Entity'][i])
                    lst_tag.append(df_final['tag'][i])
                    for i in range(len(morph_to_tokens)-1):
                        lst_tag.append(df_final['tag'][i])

                else:
                    lst_ent.append(df_final['Entity'][i])
                    lst_tag.append(df_final['tag'][i])
                    split = df_final['tag'][i].split('-')
                    for i in range(len(morph_to_tokens)-1):
                        results = 'I-' + split[-1]
                        lst_tag.append(results)

        else:
            lst_ent = result = ' '.join(str(s) for s in lst_ent)
            lst_set.append(lst_ent)
            morph_to_tokens = tokenizer.tokenize(lst_ent)
            lst_set.append(morph_to_tokens)
            lst_set.append(lst_tag)
            lst.append(lst_set)
            result = 0
            lst_ent = []
            lst_tok = []
            lst_tag = []
            lst_set = []

    with open(PATH_preprodata, 'w') as f:
        json.dump(lst, f)

    with open(PATH_preprodata, 'r') as f:
        preprodata = json.load(f)

In [13]:
class Data_NER:
    
    def make_json(file_name):
        global df_final
        PATH_data = os.path.join(PATH_dir, file_name)

        #change file extention from '.txt' to '.json'
        prepro_file_name = 'prepro_' + file_name.rsplit('.')[0] + '.json'
        PATH_preprodata = os.path.join(PATH_dir, prepro_file_name)

        with open(PATH_data, 'r', encoding='UTF8') as f:
            rawdata = f.readlines()

        # data preprocessing by using pandas
        df0 = pd.DataFrame(rawdata)

        if file_name == 'dev.txt':
            df1 = pd.DataFrame(df0[0].str.split('/').tolist(),columns=['1-1','1-2'])
            df2 = pd.DataFrame(df0[0].str.split('\t').tolist(),columns=['1','2','3','4'])
            df_final = pd.concat([df1['1-1'],df2['4'].str.strip('\n')],axis = 1)
            df_final = pd.DataFrame(df_final)
            df_final = df_final.rename(columns={'1-1':'Entity','4':'tag'})
        else:
            df1 = pd.DataFrame(df0[0].str.split('/').tolist(),columns=['1-1','1-2','1-3'])
            df2 = pd.DataFrame(df0[0].str.split('\t').tolist(),columns=['1','2','3','4'])
            df_final = pd.concat([df1['1-1'],df2['4'].str.strip('\n')],axis = 1)
            df_final = pd.DataFrame(df_final)
            df_final = df_final.rename(columns={'1-1':'Entity','4':'tag'})

        # upload tokenizer
        PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
        tokenizer = BertTokenizer.from_pretrained(PRETAINED_MODEL_NAME)

        # make dataset & tokenizer_tagging [original, token, tokenizer_tagging]
        lst=[]
        lst_set = []
        lst_ent = []
        lst_tag = []
        for i in range(len(df_final)):
            if df_final['tag'][i] != None:
                morph_to_tokens = tokenizer.tokenize(df_final['Entity'][i])
                if 1 == len(morph_to_tokens):
                    lst_ent.append(df_final['Entity'][i])
                    lst_tag.append(df_final['tag'][i])

                else:
                    if df_final['tag'][i] == 'O':
                        lst_ent.append(df_final['Entity'][i])
                        lst_tag.append(df_final['tag'][i])
                        for i in range(len(morph_to_tokens)-1):
                            lst_tag.append(df_final['tag'][i])

                    else:
                        lst_ent.append(df_final['Entity'][i])
                        lst_tag.append(df_final['tag'][i])
                        split = df_final['tag'][i].split('-')
                        for i in range(len(morph_to_tokens)-1):
                            results = 'I-' + split[-1]
                            lst_tag.append(results)

            else:
                lst_ent = result = ' '.join(str(s) for s in lst_ent)
                lst_set.append(lst_ent)
                morph_to_tokens = tokenizer.tokenize(lst_ent)
                lst_set.append(morph_to_tokens)
                lst_set.append(lst_tag)
                lst.append(lst_set)
                result = 0
                lst_ent = []
                lst_tok = []
                lst_tag = []
                lst_set = []

        with open(PATH_preprodata, 'w') as f:
            json.dump(lst, f)

        with open(PATH_preprodata, 'r') as f:
            preprodata = json.load(f)

In [12]:
make_json(file_name_ko_train)
make_json(file_name_ko_test)
make_json(file_name_ko_dev)